In [1]:
from __future__ import print_function
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import os
import torchvision
from torch.utils.tensorboard import SummaryWriter
import socket
from datetime import datetime
import scipy.io as sio   
from scipy.io import loadmat
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset
import matplotlib.pyplot as plt

In [124]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx % log_interval == 0) & (batch_idx!=0):
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct = pred.eq(target.view_as(pred)).sum().item()
            # 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            n_iter=batch_idx+(epoch-1)*len(train_loader)
            writer.add_scalar('Loss/train', loss,n_iter )
            writer.add_scalar('Accuracy/train', 100. * correct / len(target), n_iter)
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad(): #don't save gradient
        for data, target in test_loader:
            if cuda:
                data, target = data.cuda(), target.cuda()
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    n_iter=(epoch-1)
    writer.add_scalar('Loss/test', test_loss,n_iter )
    writer.add_scalar('Accuracy/test', 100. * correct / len(test_loader.dataset), n_iter)

In [125]:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
save_dir='/Users/eghbalhosseini/MyData/neural_manifolds/CNN_training_on_parition/'
data_dir='/Users/eghbalhosseini/MyData/neural_manifolds/data'
data_file='synthpartition_eh.mat'
access_rights = 0o755
try:
    os.mkdir(save_dir,access_rights)
except OSError:
    print('Creation of %s failed\n'%save_dir)

batch_size=32
test_batch_size=200 
train_set_size=5000
test_set_size=1000
epochs=10
no_cuda=False
save_epochs=False
momentum=0.7
lr=1e-5
seed=1
gamma=0.7
log_interval=150
cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

train_spec={'batch_size':batch_size,
           'train_set_size':train_set_size,
           'num_epochs':epochs,
           'lr':lr,
           'log_interval':log_interval,
           'test_batch_size':test_batch_size
           }

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}


Creation of /Users/eghbalhosseini/MyData/neural_manifolds/CNN_training_on_parition/ failed



In [126]:
class partition_dataset(Dataset):
    def __init__(self, data_dir=data_dir):
        self.data_dir=data_dir
        self.dat , self.target=self.load_data()
        self.n_samples=self.dat.shape[0]
    def __len__(self):
        return self.n_samples
    def __getitem__(self, idx):
        item=np.expand_dims(self.dat[idx],axis=0)
        targ=np.squeeze(self.target[idx])
        return torch.tensor(item,dtype=torch.float), torch.tensor(targ,dtype=torch.long)
    def load_data(self):
        annot=loadmat(self.data_dir) 
        dat=annot['data']
        dat_new=dat[:,range(28*28)]
        dat_new=np.reshape(dat_new,(-1,28,28))
        target=np.double(np.transpose(annot['class_id'])-1.0)
        return dat_new, target 

In [128]:
partition_train_dataset=partition_dataset(data_dir=os.path.join(data_dir,data_file))

train_loader = torch.utils.data.DataLoader(
    dataset=partition_train_dataset,
    batch_size=30,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset=partition_train_dataset,
    batch_size=15,
    shuffle=True)


In [129]:
# Build the neural network, expand on top of nn.Module
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 50)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [130]:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
log_dir = os.path.join(save_dir,'runs', current_time + '_' + socket.gethostname())
writer = SummaryWriter(log_dir=log_dir)

In [131]:
model = CNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
criterion = nn.CrossEntropyLoss();
#optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
#writer.add_graph(model)
writer.add_hparams(hparam_dict=train_spec,metric_dict={})


In [132]:
for epoch in range(1, epochs + 1):
    epoch_dat=train(epoch)
    print('saved epoch '+str(epoch))
    #scheduler.step()
writer.flush()
writer.close()

Train Epoch: 1 [4500/5000 (90%)]	Loss: 4.065312
saved epoch 1
Train Epoch: 2 [4500/5000 (90%)]	Loss: 3.934645
saved epoch 2
Train Epoch: 3 [4500/5000 (90%)]	Loss: 4.002067
saved epoch 3
Train Epoch: 4 [4500/5000 (90%)]	Loss: 3.991263
saved epoch 4
Train Epoch: 5 [4500/5000 (90%)]	Loss: 4.099072
saved epoch 5
Train Epoch: 6 [4500/5000 (90%)]	Loss: 3.895945
saved epoch 6
Train Epoch: 7 [4500/5000 (90%)]	Loss: 4.032368
saved epoch 7


KeyboardInterrupt: 

In [64]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()


In [60]:
target.shape

torch.Size([3])

In [72]:
epoch_target.type()

'torch.LongTensor'

In [73]:
target.type()

'torch.LongTensor'

In [133]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)
train_loader = torch.utils.data.DataLoader(
    dataset=partition_train_dataset,
    batch_size=30,
    shuffle=True)

In [ ]:
epoch_data,epoch_target = next(iteration)
optimizer.zero_grad()
output = model(epoch_data)
loss = criterion(output, epoch_target)

In [134]:
train_loader